In [1]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

// Import Semantic Kernel
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
using Azure.AI.OpenAI;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new Azure.AzureKeyCredential(apiKey);
var openAIClient = new OpenAIClient(endpoint, credentials);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12

In [2]:
var system = 
""" 
Você é um assistente para extrair informações de um texto.
""";

In [3]:
var completionOptions = new ChatCompletionsOptions
{
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f, // Top P
    DeploymentName = model
};

In [4]:
var functionDef = new FunctionDefinition()
        {
            Name = "contact-information-extraction",
            Description = "report the contact information extracted from the text",
            Parameters = BinaryData.FromObjectAsJson(
            new
            {
                Type = "object",
                Properties = new
                {
                     name = new
                    {
                        Type = "string",
                        Description = "name extracted from the text"
                    },
                    email = new
                    {
                        Type = "string",
                        Description = "email extracted from the text"
                    },
                     phone = new
                    {
                        Type = "string",
                        Description = "phone extracted from the text"
                    }
                },
                Required = new[] { "name"  },
            },
            new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }),
        };

In [5]:
completionOptions.Functions.Add(functionDef);
completionOptions.Messages.Add(new ChatRequestSystemMessage(system));

completionOptions

Azure.AI.OpenAI.ChatCompletionsOptions ChoiceCount <null> DeploymentName gpt-4 FrequencyPenalty 0 MaxTokens 400 NucleusSamplingFactor 0.95 PresencePenalty 0 StopSequences [ ] IsUndefined True Count 0 IsReadOnly False (values) [ ] Temperature 1 TokenSelectionBiases (empty) Functions [ Azure.AI.OpenAI.FunctionDefinition ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.FunctionDefinition Name contact-information-extraction Description report the contact information extracted from the text Parameters {"type":"object","properties":{"name":{"type":"string","description":"name extracted from the text"},"email":{"type":"string","description":"email extracted from the text"},"phone":{"type":"string","description":"phone extracted from the text"}},"required":["name"]} FunctionCall <null> AzureExtensionsOptions <null> ToolChoice <null> Messages [ Azure.AI.OpenAI.ChatRequestSystemMessage ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatRequestSystemMessage Content Você é um assistente para extrair informações de um texto. Name <null> Role system User <null> Seed <null> ResponseFormat <null> Tools [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty)

In [6]:
completionOptions.Messages.Add(new ChatRequestUserMessage("Olá, meu nome é João e meu email é joao@contoso.com"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name contact-information-extraction Arguments {
 "name": "João",
 "email": "joao@contoso.com"
} AzureExtensionsContext <null>

In [7]:
completionOptions.Messages.Remove(completionOptions.Messages.Last());
completionOptions.Messages.Add(new ChatRequestUserMessage( "Olá, meu nome é Maria e meu telefone é (11) 91234-5678"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name contact-information-extraction Arguments {
 "name": "Maria",
 "phone": "(11) 91234-5678"
} AzureExtensionsContext <null>

In [8]:
completionOptions.Messages.Remove(completionOptions.Messages.Last());
completionOptions.Messages.Add(new  ChatRequestUserMessage( "Olá, meu nome é Maria e meu telefone é (11) 91234-5678 e meu e-mail é maria@contoso.com"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name contact-information-extraction Arguments {
 "name": "Maria",
 "phone": "(11) 91234-5678",
 "email": "maria@contoso.com"
} AzureExtensionsContext <null>

In [9]:
completionOptions.Messages.Remove(completionOptions.Messages.Last());
completionOptions.Messages.Add(new ChatRequestUserMessage( "Maria - telefone é (11) 91234-5678 - e-mail é maria@contoso.com. \n João - telefone é (11) 91234-5678 - e-mail é joao@contoso.com"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name contact-information-extraction Arguments {
"name": "Maria",
"phone": "(11) 91234-5678",
"email": "maria@contoso.com"
} AzureExtensionsContext <null>

In [10]:
completionOptions.Messages.Add(new ChatRequestAssistantMessage(response.Choices[0].Message));
completionOptions.Messages.Add(new ChatRequestFunctionMessage("contact-information-extraction",  "OK"));

response = await openAIClient.GetChatCompletionsAsync(completionOptions);

response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name contact-information-extraction Arguments {
"name": "João",
"phone": "(11) 91234-5678",
"email": "joao@contoso.com"
} AzureExtensionsContext <null>

In [14]:
completionOptions.Messages.Add(new ChatRequestAssistantMessage(response.Choices[0].Message));
completionOptions.Messages.Add(new ChatRequestFunctionMessage("contact-information-extraction",  "OK"));

response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content I have successfully extracted the contact information for both Maria and João. ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall <null> AzureExtensionsContext <null>